## 評価指標とデータ拡張



・再現率　感度　sensitivity

真陽性　/ 真陽性＋偽陰性

再現率の向上には偽陰性を減らす。偽陽性の増加は気にしない。

・適合率　precision

真陽性　/ 真陽性＋偽陽性

適合率の改善には偽陽性を減らす。

適合率も再現性もモデルを評価する単一の評価指標には用いることができない。
学習の途中で確認することは有用

途中でどちらかの値が0に近づいているならモデルが機能していないことを意味する。

In [ ]:
def logMetrics(
            self,
            epoch_ndx,
            mode_str,
            metrics_t,
            classificationThreshold=0.5,
    ):
        self.initTensorboardWriters()
        log.info("E{} {}".format(
            epoch_ndx,
            type(self).__name__,
        ))

        negLabel_mask = metrics_t[METRICS_LABEL_NDX] <= classificationThreshold
        negPred_mask = metrics_t[METRICS_PRED_NDX] <= classificationThreshold

        posLabel_mask = ~negLabel_mask
        posPred_mask = ~negPred_mask

        neg_count = int(negLabel_mask.sum())
        pos_count = int(posLabel_mask.sum())
        # 真陽性数
        trueNeg_count = neg_correct = int((negLabel_mask & negPred_mask).sum())
        # 真陰性数
        truePos_count = pos_correct = int((posLabel_mask & posPred_mask).sum())
        # 偽陽性
        falsePos_count = neg_count - neg_correct
        # 偽陰性
        falseNeg_count = pos_count - pos_correct

真陽性数と真陰性数を使いpresisionとrecallを計算

In [ ]:
def logMetrics(
            self,
        #
        #
        #
        precision = metrics_dict['pr/precision'] = \
            truePos_count / np.float32(truePos_count + falsePos_count)
        recall    = metrics_dict['pr/recall'] = \
            truePos_count / np.float32(truePos_count + falseNeg_count)

        metrics_dict['pr/f1_score'] = \
            2 * (precision * recall) / (precision + recall)

## 最終的な評価指標：F1スコア

F1スコアは0から1の値をとり、適合率と再現性を組み合わせた方法として採用される

In [ ]:
log.info(
            ("E{} {:8} {loss/all:.4f} loss, "
                 + "{correct/all:-5.1f}% correct, "
                 + "{pr/precision:.4f} precision, "
                 + "{pr/recall:.4f} recall, "
                 + "{pr/f1_score:.4f} f1 score"
            ).format(
                epoch_ndx,
                mode_str,
                **metrics_dict,

このまま計算したら計算した値のいくつかがnanとなってエラーとなった。

一つの原因は陽性サンプルが一つも正しく分類できないことである。これは適合率と再現率が0であることを意味している。
二つ目は検証において陽性として分類されたサンプルが無いのでtrueposとふfalseposのカウントが0となっている点である。

訓練中はこのように陽性サンプルが少なくても訓練の前半ではパラメータがランダムなので、陽性と判断する場合はある。
訓練後の検証では陽性と判断されなくなってしまいエラーとなった。

##　理想的なデータセット

モデルを適切に訓練するためのはデータのバランスを取る必要があることが示された。

今回はモデルは十分は分類能力があるが、データが偏っている。（400：1）
最終的に目指したいのはこのような偏りがあるデータをモデルが適切に処理できるようにすることである。

途方もないエポック数をかければ訓練できるかもしれない。現実的にはデータのバランスを取る。

## 陽性サンプルが少ないと影響はかき消される。

予測結果が正解ラベルに近いとネットワークの重みは大きく変化しない。一方予測結果と正解がかけ離れている場合は重みの値を大きう変化させる。そして陽性サンプルが訓練されるまでに多くのバッチを学習している場合はパラメータは陰性を予測する側に偏っている。


## 解決策1：サンプラの使用

Dataloaderのオプション引数のsamplerは特定のサンプルの抽出を制限したり、複数回抽出したりできる。
しかしこれはDatasetのサブクラスにサンプルのインデックスを出力する関数が必要になる。

## 解決策2：
Datasetのサブクラス内でデータの再構成をする。陰性と陽性のサンプルのリストを作成し、交互にサンプルが取り出されるようにする。

In [ ]:
class LunaDataset(Dataset):
    def __init__(
        self,
        val_stride=0,
        isValSet_bool=None,
        series_uid=None,
        sortby_str="random",
        ratio_int=0,
        augmentation_dict=None,
        candidateInfo_list=None,
    ):
        # ratio_intが0なら元のバランスのまま
        self.ratio_int = ratio_int
        #
        #
        # 結節と非結節のリスト作成
        # candidateInfo_listには全ての結節の位置、boolearnの情報がある
        self.negative_list = [nt for nt in self.candidateInfo_list if not nt.isNodule_bool]
        self.pos_list = [nt for nt in self.candidateInfo_list if nt.isNodule_bool]
        #
        #
        #　リストをシャッフル
        def shuffleSamples(self):
        if self.ratio_int:
            random.shuffle(self.negative_list)
            random.shuffle(self.pos_list)
        #
        #
        #　
        def __getitem__(self, ndx):
        if self.ratio_int:
            pos_ndx = ndx // (self.ratio_int + 1)

            if ndx % (self.ratio_int + 1):
                # 念のため非結節も
                neg_ndx = ndx - 1 - pos_ndx
                neg_ndx %= len(self.negative_list)
                candidateInfo_tup = self.negative_list[neg_ndx]
            else:
                # 結節のデータは少ないため、ndxの数が大きくなった場合にオーバーフローしないようにしている
                pos_ndx %= len(self.pos_list)
                candidateInfo_tup = self.pos_list[pos_ndx]
        else:
            candidateInfo_tup = self.candidateInfo_list[ndx]

        width_irc = (32, 48, 48)

In [28]:
a = 5020
a %= 100
a

20

In [29]:
b = 5021
b %= 100
b

21

ratio_intが2の場合は2：1で陰性：陽性として取り出す。
10個取り出すndx=10なら、　pos_ndxは10//3=3

サンプルのバランスを取ることで実際のデータ数の意味がない。またデータ数が多ければ同じ結節が選ばれる頻度も増えるため、
__len__の数を決めてしまう。

In [ ]:
def __len__(self):
        if self.ratio_int:
            return 200000
        else:
            return len(self.candidateInfo_list)

appのコマンドライン引数にもこの調整の引数入力を追加。

In [ ]:
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]
#
#
#
        parser.add_argument('--balanced',
            help="Balance the training data to half positive, half negative.",
            action='store_true',
            default=False,

訓練セットでは問題なく、検証では成績が低下傾向にあれば過学習を疑う。

検証でも陽性的中率が70はあるので、ある程度の汎化性能はあると考える。

## データオーギュメンテーション